In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-03-14 20:20:44--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.08MB/s    in 0.2s    

2021-03-14 20:20:45 (6.08 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Shoes_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18069663|R3P2HIOQCIN5ZU|B000XB31C0|     265024781|Minnetonka Men's ...|           Shoes|          1|            0|          0|   N|                Y|                   .|Do not buy: reall...| 2015-08-31|
|         US|   16251825|R12VVR0WH5Q24V|B00CFYZH5W|     259035853|Teva Men's Pajaro...|           Shoes|          5|    

In [58]:
#Create the Vine Table
vine_df = df.select(["review_id", "star_rating", "helpful_votes","total_votes", "vine", "verified_purchase"])
vine_one = vine_df.count()

4366916

In [65]:
#Filter Vine to show total votes greater than 20
#total_votes_df = vine_df.filter("total_votes>=20").show()
total_votes_df = vine_df.filter("total_votes>=20")
#print(total_votes_df.count())
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [66]:
#Filter Total Votes to show where helpful votes divided by total votes is = to or > than 50%
helpful_votes = total_votes_df.filter("helpful_votes / total_votes >0.5")
helpful_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

In [92]:
# Filter to show rows where a review was written as part of the Vine program
#verified_purchase = helpful_votes.filter("verified_purchase == 'Y'").show()
Vine = helpful_votes.filter("vine == 'Y'")
Vine.show()
Vine.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2N45ZKRRZS856|          5|           21|         22|   Y|                N|
| R5OMLMK13A8NS|          5|           34|         38|   Y|                N|
|R2MPEQ4SPTEQNS|          4|          180|        184|   Y|                N|
| RIR0D3KJ0CQ31|          4|           21|         21|   Y|                N|
|R1SPWJDHUWWC5E|          5|           88|         98|   Y|                N|
|R1X6M5XA3FT98W|          5|           24|         26|   Y|                N|
|R37VCW6HA0Z72T|          5|           27|         28|   Y|                N|
|R2XRYNV2SY3ZKL|          5|           53|         56|   Y|                N|
|R1Y93KWKAX1P5N|          2|           26|         31|   Y|                N|
|R2QVTDYYLTP8SL|          5|           21|         24|   Y|     

22

In [93]:
# Filter to show rows where a review was NOT written as part of the Vine program
Non_vine = helpful_votes.filter("vine == 'N'")
Non_vine.show()
Non_vine.count()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R37F42INKX7L9K|          5|           45|         49|   N|                Y|
|R2EHKYNEP8WVSR|          5|           25|         25|   N|                Y|
| RXOS7BHID0UHL|          5|           16|         27|   N|                N|
|R33HHGFPB403GM|          5|           19|         21|   N|                Y|
| RY9O9XNLP464N|          2|           19|         22|   N|                Y|
|R2VP11C28JAEZF|          5|           30|         30|   N|                Y|
|R1TXGR1HA2M3P3|          5|           28|         29|   N|                Y|
| R6OD85TMEMHQQ|          5|           28|         28|   N|                Y|
|R1G4PAJXP3FTN7|          2|           43|         51|   N|                Y|
|R2P2S8UGUMCOLX|          5|           21|         22|   N|     

26971

In [75]:
#Total number of reviews
total_reviews = vine_df.count()
print(total_reviews)

4366916


In [76]:
#Total number reviews after filter 
total_filtered_votes = total_votes_df.count()
print(total_filtered_votes)

27884


In [77]:
#Total number of helpful reviews
helpful_reviews = helpful_votes.count()
print(helpful_reviews)

26993


In [78]:
#Total 5 star reviews
vine_total_five_star_reviews = vine_df.filter("star_rating == 5").count()
print(vine_total_five_star_reviews)

2639935


In [79]:
#Total 5 star reviews
total_five_star_reviews = total_votes_df.filter("star_rating == 5").count()
print(total_five_star_reviews)

14594


In [86]:
#5 star reviews Vine program
five_star_reviews_vine_program = Vine.filter("star_rating == 5").count()
print(five_star_reviews_vine_program)

13


In [87]:
#5 star reviews NON Vine program
five_star_reviews_non_vine_program = Non_vine.filter("star_rating == 5").count()
print(five_star_reviews_non_vine_program)

14475


In [90]:
#percentage of 5 star reviews Vine program
vine_program_percentage = (five_star_reviews_vine_program / Vine.count() ) * 100
print(vine_program_percentage)

59.09090909090909


In [91]:
#percentage of 5 star reviews Non Vine program
non_vine_program_percentage = (five_star_reviews_non_vine_program / Non_vine.count()) * 100
print(non_vine_program_percentage)

53.66875532979867
